In [0]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
U = [[1,4,2,2],[5,2,4,4],[np.nan,np.nan,3,np.nan],[2,5,np.nan,5],[4,1,np.nan,1],[np.nan,2,5,np.nan]]
U = pd.DataFrame(U)
U.columns = ['Ken','Lee','Meg','Nan']
U.index = ['A','B','C','D','E','F']
U

In [0]:
pear = U.corr()

In [0]:
pear

In [0]:
mean = U.mean()

In [0]:
mean

In [0]:
ken_mean = mean[0]
ken_mean

In [0]:
U.iloc[5,:]

In [0]:
round(U.mean().mean())

In [0]:
pear.iloc[0,:]

In [0]:
(U.iloc[5,:] - round(U.mean().mean()))*pear.iloc[0,:]

In [0]:
bunsi = ((U.iloc[5,:] - round(U.mean().mean()))*pear.iloc[0,:]).sum()
bunsi

In [0]:
bumbo = pear.iloc[0,].abs().sum()-1
bumbo

In [0]:
ken_mean +  bunsi/bumbo

In [0]:
ls

In [0]:
rating = pd.read_csv('./ratings.csv')
movie = pd.read_csv('./movies.csv')

In [0]:
rating.info()

In [0]:
rating.head()

In [0]:
rating.describe()

In [0]:
rating['movieId'].nunique()

In [0]:
rating['userId'].nunique()

In [0]:
movie.info()

In [0]:
movie.head()

In [0]:
movie.describe()

In [0]:
df = pd.merge(rating,movie,on='movieId',how='left')
df.head()

In [0]:
data = df[['userId','title','rating']]
data.head()

In [0]:
data.groupby('title')['userId'].count().sort_values(ascending=False).head()

In [0]:
data.groupby('title')['userId'].count().sort_values(ascending=False).tail()

In [0]:
title_rating = data.groupby('title')['userId'].count().sort_values(ascending=False)

In [0]:
plt.figure(figsize=(15,10))
title_rating.plot()
plt.grid()

In [0]:
data.groupby('userId')['userId'].count().sort_values(ascending=False).head()

In [0]:
data.groupby('userId')['userId'].count().sort_values(ascending=False).tail()

In [0]:
plt.figure(figsize=(15,10))
sb.barplot(x='rating', y='userId',data=data)

In [0]:
df = pd.pivot_table(data, index='userId',columns='title')
df.head()

In [0]:
plt.figure(figsize=(18,18))
sb.heatmap(df,cmap='Blues')

In [0]:
test = pd.pivot_table(data,index='userId',columns='title',aggfunc='sum',margins=True)
test.tail()

In [0]:
test.columns[0]

In [0]:
test = test.sort_values(by='All',ascending=False,axis=1).sort_values(by=('rating','All'),ascending=False)

In [0]:
plt.figure(figsize=(18,18))
sb.heatmap(test)

In [0]:
toy = test.iloc[50:70,50:100].T
toy.head()

In [0]:
corrs = toy.corr()
corrs.head()

In [0]:
corrs.loc[:,357].head()

In [0]:
user_corr = corrs.loc[:,357].sort_values(ascending=False)
user_corr.head()

In [0]:
user_corr_top5 = user_corr[1:6]
user_corr_top5

In [0]:
user_corr_top5.index

In [0]:
rating_top5 = toy.loc[:,user_corr_top5.index]
rating_top5.head()

In [0]:
usermean = rating_top5.mean()

In [0]:
normratings = rating_top5 - usermean

In [0]:
normratings.mean()

In [0]:
weighted_ratings = (normratings*user_corr_top5).sum(axis=1)

In [0]:
normratings.notnull().head()

In [0]:
toy.loc[:,357].mean()

In [0]:
sumweight = (normratings.notnull()*user_corr_top5).sum(axis=1)

In [0]:
predict = toy.loc[:,357].mean() + weighted_ratings/sumweight

In [0]:
predict.sort_values(ascending=False)

In [0]:
toy = toy.T
toy.head()

In [0]:
itemmean = toy.mean()
itemmean.head()

In [0]:
itemmean[0]

In [0]:
normrating = (toy-itemmean)
normrating.head()

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cosine = pd.DataFrame(cosine_similarity(normrating.T.fillna(0)))
cosine.index  = normrating.columns
cosine.columns = normrating.columns

In [0]:
cosine.head()

In [0]:
user_rating = normrating.loc[357,:]

In [0]:
pirates_sim = cosine.iloc[0]

In [0]:
weighted_ratings = (user_rating*pirates_sim).sum()

In [0]:
sumweight = (user_rating.notnull()*pirates_sim.abs()).sum()

In [0]:
itemmean[0]+weighted_ratings/sumweight

In [0]:
weighted_ratings = (user_rating*cosine).sum(axis=1)
sumweight = (user_rating.notnull()*cosine.abs()).sum(axis=1)

In [0]:
predict = itemmean + weighted_ratings/sumweight
predict.head()

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
knn = NearestNeighbors(metric='cosine',n_neighbors=50)
knn.fit(normrating.T.fillna(0))
distance,indices = knn.kneighbors(normrating.T.fillna(0))

In [0]:
distance[0]

In [0]:
indices[0]

In [0]:
distance_0 = pd.DataFrame(1-distance[0])
distance_0.index = normrating.iloc[:,indices[0]].columns
distance_0.head()

In [0]:
nume = (normrating.iloc[6, indices[0]]*(1-distance[0])).sum()

In [0]:
deno = ((normrating.iloc[6,indices[0]].notnull())*(np.abs(1-distance[0]))).sum()

In [0]:
user357mean = toy.iloc[:,indices[0][0]].mean()

In [0]:
user357mean + nume /deno

In [0]:
d = np.zeros(distance.shape[0])

In [0]:
for i in range(distance.shape[0]):
  a = (normrating.iloc[6, indices[i]]*(1-distance[i])).sum()
  b = ((normrating.iloc[6,indices[i]].notnull())*(np.abs(1-distance[i]))).sum()
  c = toy.iloc[:,indices[i][0]].mean()
  d[i] = c + a/b

In [0]:
d = pd.DataFrame(d)
d.index = toy.columns
d.head()